# Tutorial:  PyOphidia command examples

First of all import PyOphidia modules

In [ ]:
from PyOphidia import cube, client

**PyOphidia** is a GPLv3-licensed Python module to interact with the Ophidia framework. It implements two main classes:
   
- **Client** class: it supports the submissions of Ophidia commands and workflows as well as the management of sessions from Python code
- **Cube** class: it builds on the client class and provides the datacube type abstraction and the methods to manipulate, process and get information on cubes objects
   
While the *cube* module provides a user-friendly interface, the *client* module allows a finer specification of the operators.

As a first command we need to connect to the Ophidia server front-end to load the modules variables and start an analytics session (connection details are inferred from the environment with ```read_env=true```).

In [ ]:
cube.Cube.setclient(read_env=True)

Let's now load a NetCDF file. We can inspect the file with the *explorenc* Ophidia operator

In [ ]:
cube.Cube.explorenc(
            src_path="/home/ophidia/notebooks/tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc"
        )

We can now create a datacube from the NetCDF file:
- The file path is **/home/ophidia/notebooks/tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc**
- The variable to be imported is **tasmax**
- Data should be arranged in order to operate on time series (```imp_dim='time'```) 

**Note: We are not directly reading the file content from the notebook**

**Single core**: Import the input NetCDF file using 1 core

In [ ]:
%%time
mycube = cube.Cube.importnc2(
            src_path='/home/ophidia/notebooks/tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc',
            measure='tasmax',
            imp_dim='time',
            ncores=1,
            description="Imported cube (1 core)"
        )

**Multi-core**: Import the input NetCDF file using 4 cores

In [ ]:
%%time
mycube = cube.Cube.importnc2(
            src_path='/home/ophidia/notebooks/tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc',
            measure='tasmax',
            imp_dim='time',
            ncores=4,
            description="Imported cube (4 cores)",
        )

Check the datacubes available in the virtual file system. Ophidia manages a virtual file system associated with each user that provides a hierarchical organization of concepts, supporting: 

* *datacubes*, the actual objects containing the data and related metadata;
* *containers*, grouping together a set of related datacubes; 
* *virtual folders*, to store one or more containers according to the user's needs. 

In [ ]:
cube.Cube.list(level=2)

To get the list of arguments and default values we can use the python *help()* command can be used

In [ ]:
help(cube.Cube.list)

Inspect the cube and its dimensions structure. Note the data fragmentation table

In [ ]:
mycube.info()

Subset the datacube over space (lat and lon) and time. A filter with the actual dimension values  can be provided using ```subset_type="coord"```.

**Note: each instance method produces a new datacube object**

In [ ]:
mycube2 = mycube.subset(
            subset_dims="lat|lon|time",
            subset_filter="-50:20|20:160|150:240",
            subset_type="coord",
            ncores=2,
            description="Subsetted cube"
        )

Inspect the new cube: the dimension sizes and fragmentation info have changed

In [ ]:
mycube2.info()

But what does the datacube actually contain at this point? We can use the ```explore``` method to check the content. 

In [ ]:
mycube2.explore(limit_filter=1)

We can also explore just a specific portion of the datacube. ```explore``` supports filters on multiple dimensions similarly to the subset method 

In [ ]:
mycube2.explore(subset_dims="lat|lon|time",subset_type="index",subset_filter="1:2|1:4|1:4")

Let's compute the **maximum** value over the time series for each point in the spatial domain. We can also compute other metrics (see http://ophidia.cmcc.it/documentation/users/operators/OPH_REDUCE.html)

In [ ]:
mycube3 = mycube2.reduce(
                    operation='max',
                    ncores=2,
                    description="Reduced cube"
                )

In the new cube the time dimension is "collapsed" (size: *ALL*)

In [ ]:
mycube3.info()

We can now reorganize the data structure by making the longitude dimension an array-oriented dimension

In [ ]:
mycube4 = mycube3.rollup(
                ncores=2,
                description="Rollup cube"
          )

The new cube will now have *lon* as an array-dimension

In [ ]:
mycube4.info()

Each operation executed creates a new datacube on the framework (datacubes are not overwritten)

In [ ]:
cube.Cube.list(level=2)

Let's export the data into a Python-friendly structure. 

**Note: this is the first time we move data from the server-side to the Notebook**

The structure looks something like this

<img src="imgs/export_array.png" alt="Export Array" width="800">



In [ ]:
data = mycube4.export_array()

from IPython.lib.pretty import pprint
pprint(data)

The data exported in the Python structure can be used to create a map (note the definition of a Python function)

In [ ]:
%matplotlib inline

def plotData(data):
    
    import cartopy.crs as ccrs
    import matplotlib.pyplot as plt
    from cartopy.mpl.geoaxes import GeoAxes
    from cartopy.util import add_cyclic_point
    import numpy as np
    import warnings
    warnings.filterwarnings("ignore")

    fig = plt.figure(figsize=(12, 6), dpi=100)

    #Add Geo axes to the figure with the specified projection (PlateCarree)
    projection = ccrs.PlateCarree()
    ax = plt.axes(projection=projection)

    #Draw coastline and gridlines
    ax.coastlines()

    gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
    gl.xlabels_top = False
    gl.ylabels_right = False

    lat = data['dimension'][0]['values'][ : ]
    lon = data['dimension'][1]['values'][ : ]
    var = data['measure'][0]['values'][ : ]
    var = np.reshape(var, (len(lat), len(lon)))

    #Wraparound points in longitude
    var_cyclic, lon_cyclic = add_cyclic_point(var, coord=np.asarray(lon))
    x, y = np.meshgrid(lon_cyclic,lat)

    #Define color levels for color bar
    clevs = np.arange(200,340,5)

    #Set filled contour plot
    cnplot = ax.contourf(x, y, var_cyclic, clevs, transform=projection,cmap=plt.cm.jet)
    plt.colorbar(cnplot,ax=ax)

    ax.set_aspect('auto', adjustable=None)

    plt.title('Maximum Near-Surface Air Temperature (deg K)')
    plt.show()
    
plotData(data)

We can save the results in a NetCDF file

In [ ]:
mycube4.exportnc2(
    output_path="/home/ophidia/notebooks",
    output_name="max"
)

#### What If we want to consider the whole spatial domain and specify a subset only on the time range? 

We can perform the new set of operations on *mycube* object, without the need to re-import the dataset from the file. The time range can be provided in human-readable form with a datetime format setting ```time_filter="yes"```.

In [ ]:
newMycube2 = mycube.subset(
                subset_dims="time",
                subset_filter="2096-01-01_2096-12-31",
                subset_type="coord",
                time_filter="yes",
                ncores=2,
                description="New subsetted cube"
        )

newMycube2.info()

We can the rerun the same operations on the new cube ...

In [ ]:
newMycube3 = newMycube2.reduce(
                operation='max',
                ncores=2,
                description="New reduced cube"
            )

newMycube4 = newMycube3.rollup(
                ncores=2,
                description="New rollup cube"
            )

... and plot the new datacube values on a map using the function *plotData*

In [ ]:
data = newMycube4.export_array()
plotData(data)

#### What if we want to get the *minimum* instead of the maximum value?

Again we can perform the new set of operations on *newMycube2* object, without the need to re-import or subset the dataset again

In [ ]:
newNewMycube3 = newMycube2.reduce(
                    operation='min',
                    ncores=2,
                    description="New reduced cube2"
                )

newNewMycube4 = newNewMycube3.rollup(
                    ncores=2,
                    description="New rollup cube2"
                )

... and plot the new datacube values on a map using the function *plotData*

In [ ]:
data = newNewMycube4.export_array()
plotData(data)

We can use the ***predicate*** evaluation operation to identify the days with temperature below a given threshold, e.g. 273.15°K (see http://ophidia.cmcc.it/documentation/users/primitives/OPH_PREDICATE.html).

In [ ]:
icingdays = mycube.apply(
    query="oph_predicate('OPH_FLOAT','OPH_INT',measure,'x-273.15','<0','1','0')"
)

and count days below the given threshold on yearly basis (this is known as the *Icing Days* climate index)

In [ ]:
count = icingdays.reduce2(
    operation='sum',
    dim='time',
    concept_level='y',
)
count.info()

Let's plot the first year from the last cube

In [ ]:
firstyear = count.subset(subset_filter=1, subset_dims='time')

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.util import add_cyclic_point
import numpy as np
import warnings
warnings.filterwarnings("ignore")

fig = plt.figure(figsize=(15, 6), dpi=100)

#Add Geo axes to the figure with the specified projection (PlateCarree)
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)

#Draw coastline and gridlines
ax.coastlines()

gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
gl.xlabels_top = False
gl.ylabels_right = False

data = firstyear.export_array(show_time='yes')
lat = data['dimension'][0]['values'][ : ]
lon = data['dimension'][1]['values'][ : ]
var = data['measure'][0]['values'][ : ]
var = np.reshape(var, (len(lat), len(lon)))

#Wraparound points in longitude
var_cyclic, lon_cyclic = add_cyclic_point(var, coord=np.asarray(lon))
x, y = np.meshgrid(lon_cyclic,lat)

#Define color levels for color bar
levStep = (np.nanmax(var)-np.nanmin(var))/20
clevs = np.arange(np.nanmin(var),np.nanmax(var)+levStep,levStep)

#Set filled contour plot
cnplot = ax.contourf(x, y, var_cyclic, clevs, transform=projection,cmap=plt.cm.Blues)
plt.colorbar(cnplot,ax=ax)

ax.set_aspect('auto', adjustable=None)

plt.title('Icing Days (year 2096)')
plt.show()

#### Time series processing

Starting from the first imported datacube, we can extract a single time series for a given spatial point

In [ ]:
mycube2 = mycube.subset(
            subset_dims="lat|lon|time",
            subset_filter="42|15|2096-01-01_2096-12-31",
            subset_type="coord",
            ncores=2,
            time_filter="yes",
            description="Subsetted cube"
)

Then compute the moving average on each element of the measure array (see http://ophidia.cmcc.it/documentation/users/primitives/OPH_MOVING_AVG.html)

In [ ]:
moving_avg = mycube2.apply(
    query="oph_moving_avg('OPH_FLOAT','OPH_FLOAT',measure,7.0,'OPH_SMA')"
)

and plot the two time series (*mycube2* and *moving_avg*)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

data = mycube2.export_array()
data_mov = moving_avg.export_array()

y1 = data['measure'][0]['values'][0][:]
y2 = data_mov['measure'][0]['values'][0][:]
x = data['dimension'][2]['values'][:]
plt.figure(figsize=(11, 3), dpi=100)

plt.plot(x, y1,'r',label="2096")
plt.plot(x, y2,'g',label="moving_avg")
plt.legend(loc="upper left")

plt.ylabel(data['measure'][0]['name'] + " (degK)")

plt.xlabel("Days since 2096/01/01")
plt.title('Maximum Near-Surface Air Temperature')
plt.show()

**Compare two time series.**

We can also compute the difference between two time series (also from different cubes). 

Let's first compute the average value over the time series

In [ ]:
avgCube = mycube.reduce2(
    operation='avg',
    dim='time',
    concept_level='M',
)

Extract the first time series (2096)

In [ ]:
firstYear = avgCube.subset(
                subset_dims="lat|lon|time", 
                subset_type="coord", 
                subset_filter="42|15|2096-01-01_2096-12-31",
                ncores=2,
                time_filter="yes",
                description="Subsetted cube (2096)"
            )

Extract the second time series (2097)

In [ ]:
secondYear = avgCube.subset(
                subset_dims="lat|lon|time", 
                subset_type="coord", 
                subset_filter="42|15|2097-01-01_2097-12-31",
                ncores=2,
                time_filter="yes",
                description="Subsetted cube (2097)"
            )

We can check the strcuture for the 2nd cube

In [ ]:
secondYear.info()

Compute the difference between 2097 and 2096 monthly-mean time series. The **intercube** operator provides other types of inter-cube operations (http://ophidia.cmcc.it/documentation/users/operators/OPH_INTERCUBE.html)

In [ ]:
diff = secondYear.intercube(cube2=firstYear.pid,operation="sub")

and finally plot the result

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

data = diff.export_array()

y = data['measure'][0]['values'][0][:]
x = data['dimension'][2]['values'][:]
plt.figure(figsize=(11, 5), dpi=100)
plt.grid(zorder=0)
plt.bar(x, y, width=10, zorder=2)

plt.ylabel(data['measure'][0]['name'] + " difference (degC)")
plt.title('Maximum Near-Surface Air Temperature - difference 2097-2096')
plt.xticks(x, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation='vertical')
plt.show()

Our workspace now contains several datacubes from the experiments just run. Once done, we can clear the space before moving to other notebooks. 

In [ ]:
cube.Cube.deletecontainer(container='tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc',force='yes')

The virtual file system should now be "clean"

In [ ]:
cube.Cube.list(level=2)